# Load data

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# Database connection parameters
db_user = 'postgres'
db_password = 'Asmara2204'
db_host = 'localhost'
db_port = '5432'
db_name = 'abc_corp'

In [4]:
# Create a database connection
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [5]:
# Load the Excel data into a DataFrame
file_path = '../hr-dataset.xlsx'
df = pd.read_excel(file_path, sheet_name='HR Data')

In [11]:
# department dataframe
departments_df = df[['DEPARTMENT']].drop_duplicates()
departments_df.columns = departments_df.columns.str.lower()
departments_df.head()

,department
0,Distribution
6,HQ
12,IT
18,Product Development
27,Sales


In [12]:
# insert data into department table
departments_df.to_sql('department', engine, if_exists='append', index=False)

5

In [16]:
# education dataframe
education_df = df[['EDUCATION LEVEL']].drop_duplicates()
education_df.columns = ['education_level']
education_df.head()


,education_level
0,No College
1,Some College
2,Associates Degree
17,Bachelors Degree
35,Masters Degree


In [17]:
# insert data into education table
education_df.to_sql('education', engine, if_exists='append', index=False)

7

In [19]:
# job dataframe
job_df = df[['JOB_TITLE']].drop_duplicates()
job_df.columns = job_df.columns.str.lower()
print(job_df)

                    job_title
0    Administrative Assistant
30     Database Administrator
38            Design Engineer
50              Legal Counsel
68                    Manager
72           Network Engineer
93                  President
94                  Sales Rep
156    Shipping and Receiving
171         Software Engineer


In [20]:
# insert data into job table
job_df.to_sql('job', engine, if_exists='append', index=False)

10

In [24]:
# location dataframe
locations_df = df[['LOCATION', 'ADDRESS', 'CITY', 'STATE']].drop_duplicates(subset=['LOCATION', 'ADDRESS', 'CITY', 'STATE'])
locations_df.columns = locations_df.columns.str.lower()
print(len(locations_df))

5


In [22]:
# insert data into location table
locations_df.to_sql('location', engine, if_exists='append', index=False)

5

In [52]:
# manager dataframe
manager_df = df[df['MANAGER'].notnull()][['MANAGER']].drop_duplicates()
manager_df = df[df['EMP_NM'].isin(manager_df['MANAGER'])][['EMAIL', 'EMP_NM', 'DEPARTMENT']].drop_duplicates()
manager_df.columns = manager_df.columns.str.lower()
manager_df = manager_df.merge(departments_df, on='department', how='left')





In [54]:
# read department table to a dataframe to get department_id
department_df = pd.read_sql_table('department', engine)
manager_df = manager_df.merge(department_df, left_on='department', right_on='department', how='left')
manager_df = manager_df[['email', 'emp_nm', 'department_id']]
manager_df.columns = ['email', 'manager_name', 'department_id']
print(manager_df)

                               email          manager_name  department_id
0        Allison.Gentle@TechCorp.com        Allison Gentle              1
1          Jacob.Lauber@TechCorp.com          Jacob Lauber              3
2          Conner.Kinch@TechCorp.com          Conner Kinch              4
3  Jennifer.De La Garza@TechCorp.com  Jennifer De La Garza              5
4      Tyrone.Hutchison@TechCorp.com      Tyrone Hutchison              2


In [55]:
# insert data into manager table
manager_df.to_sql('manager', engine, if_exists='append', index=False)

5

In [101]:
# employee dataframe
employee_df = df[['EMP_ID', 'EMP_NM', 'EMAIL', 'HIRE_DT', 'JOB_TITLE', 'DEPARTMENT', 'MANAGER', 'EDUCATION LEVEL']].copy()

In [102]:
employee_df.drop_duplicates(inplace=True)
employee_df.columns = employee_df.columns.str.lower()
employee_df = employee_df.rename(columns={'education level': 'education_level', 'manager': 'manager_name'})
employee_df.head()

,emp_id,emp_nm,email,hire_dt,job_title,department,manager_name,education_level
0,E17469,Haifa Hajiri,Haifa.Hajiri@TechCorp.com,2003-12-17,Administrative Assistant,Distribution,Allison Gentle,No College
1,E27621,Wendell Mobley,Wendell.Mobley@TechCorp.com,2013-11-27,Administrative Assistant,Distribution,Allison Gentle,Some College
2,E27909,Michael Sperduti,Michael.Sperduti@TechCorp.com,2014-06-20,Administrative Assistant,Distribution,Allison Gentle,Associates Degree
3,E35053,Ashley Bergman,Ashley.Bergman@TechCorp.com,2009-03-01,Administrative Assistant,Distribution,Allison Gentle,No College
4,E51723,Carlos Lopez,Carlos.Lopez@TechCorp.com,2014-05-23,Administrative Assistant,Distribution,Allison Gentle,No College


In [103]:
# extract the details for the email Toni.Lembeck@TechCorp.com
print(employee_df[employee_df['email'] == 'Toni.Lembeck@TechCorp.com'])


     emp_id        emp_nm                      email    hire_dt  \
32   E27498  Toni Lembeck  Toni.Lembeck@TechCorp.com 2001-07-18   
201  E27498  Toni Lembeck  Toni.Lembeck@TechCorp.com 2001-07-18   

                  job_title department  manager_name   education_level  
32   Database Administrator         IT  Jacob Lauber  Bachelors Degree  
201        Network Engineer         IT  Jacob Lauber  Bachelors Degree  


In [104]:
# read job table, department table, manager table, and education table to a dataframe to get job_id, department_id, manager_id, and education_id
job_df = pd.read_sql_table('job', engine)
department_df = pd.read_sql_table('department', engine)
manager_df = pd.read_sql_table('manager', engine)
education_df = pd.read_sql_table('education', engine)

# Rename the conflicting column in department_df

employee_df = employee_df.merge(job_df, left_on='job_title', right_on='job_title', how='left', suffixes=('', '_job'))
employee_df = employee_df.merge(department_df, left_on='department', right_on='department', how='left', suffixes=('', '_department'))
employee_df = employee_df.merge(manager_df, left_on='manager_name', right_on='manager_name', how='left', suffixes=('', '_manager'))
employee_df = employee_df.merge(education_df, left_on='education_level', right_on='education_level', how='left', suffixes=('', '_education'))

employee_df = employee_df[['emp_id', 'emp_nm', 'email', 'hire_dt', 'job_title_id', 'department_id', 'manager_id', 'education_id']]



In [105]:
employee_df.columns

Index(['emp_id', 'emp_nm', 'email', 'hire_dt', 'job_title_id', 'department_id',
       'manager_id', 'education_id'],
      dtype='object')

In [106]:
employee_df.head()

,emp_id,emp_nm,email,hire_dt,job_title_id,department_id,manager_id,education_id
0,E17469,Haifa Hajiri,Haifa.Hajiri@TechCorp.com,2003-12-17,1,1,1.0,1
1,E27621,Wendell Mobley,Wendell.Mobley@TechCorp.com,2013-11-27,1,1,1.0,2
2,E27909,Michael Sperduti,Michael.Sperduti@TechCorp.com,2014-06-20,1,1,1.0,3
3,E35053,Ashley Bergman,Ashley.Bergman@TechCorp.com,2009-03-01,1,1,1.0,1
4,E51723,Carlos Lopez,Carlos.Lopez@TechCorp.com,2014-05-23,1,1,1.0,1


In [107]:
# extract the details for the email Toni.Lembeck@TechCorp.com
print(employee_df[employee_df['email'] == 'Toni.Lembeck@TechCorp.com'])

     emp_id        emp_nm                      email    hire_dt  job_title_id  \
32   E27498  Toni Lembeck  Toni.Lembeck@TechCorp.com 2001-07-18             2   
201  E27498  Toni Lembeck  Toni.Lembeck@TechCorp.com 2001-07-18             6   

     department_id  manager_id  education_id  
32               3         2.0             4  
201              3         2.0             4  


In [108]:
# insert data into employee table
employee_df.to_sql('employee', engine, if_exists='append', index=False)

205

In [6]:
# salary dataframe
salary_df = df[['EMP_ID', 'SALARY', 'START_DT', 'END_DT']].copy()
salary_df.columns = salary_df.columns.str.lower()

salary_df.head()


,emp_id,salary,start_dt,end_dt
0,E17469,47418,2003-12-17,NaT
1,E27621,28969,2013-11-27,NaT
2,E27909,43778,2014-06-20,NaT
3,E35053,41090,2009-03-01,NaT
4,E51723,35825,2014-05-23,NaT


In [7]:
# insert data into salary table
salary_df.to_sql('salary', engine, if_exists='append', index=False)

205